In [7]:
import requests
import pandas as pd

# Your API key and stats data ID
app_id = "6f9fe368221c798101826ea9851d0c4fe2c25eb6"
stats_data_id = "0003006031"

# Construct the URL
url = (
    f"http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData?appId={app_id}"
    f"&lang=E&statsDataId={stats_data_id}"
    "&metaGetFlg=Y&cntGetFlg=N&explanationGetFlg=Y"
    "&annotationGetFlg=Y&sectionHeaderFlg=1&replaceSpChars=0"
)

try:
    # Make the API request
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    # Extract the class information to map codes to names
    class_inf = data["GET_STATS_DATA"]["STATISTICAL_DATA"]["CLASS_INF"]["CLASS_OBJ"]
    industry_map = {}
    
    for item in class_inf:
        # Identify the industry classification block
        if item["@id"] == "cat04":
            for sub_item in item["CLASS"]:
                industry_map[sub_item["@code"]] = sub_item["@name"]
    
    # Extract data values
    records = data["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"]
    
    # Convert to DataFrame
    df = pd.DataFrame(records)

    # Map industry codes to names
    df["Industry_EN"] = df["@cat04"].map(industry_map)

    # Keep only relevant columns
    df = df[["@cat04", "Industry_EN", "$", "@time", "@cat01", "@cat03"]]
    df.columns = ["Industry_Code", "Industry_EN", "Value", "Time", "Sex", "Organization_Type"]

    # Filter for March 2025 entries
    df = df[df["Time"] == "2023001010"]

    # Save to CSV
    df.to_csv("japan_labor_2025_march.csv", index=False, encoding='utf-8')
    print("Data saved to japan_labor_2025_march.csv")
    
except requests.RequestException as e:
    print("Request failed:", e)
except KeyError as e:
    print("Unexpected data format:", e)


Data saved to japan_labor_2025_march.csv


In [11]:
# Define the regrouping mapping
industry_mapping = {
    "Mining and quarrying of stone and gravel": ["Mining and quarrying of stone and gravel"],
    "Construction": ["Construction"],
    "Manufacturing": [
        "Manufacturing", "Manufacture of food", "Manufacture of beverages, tobacco and feed",
        "Manufacture of textile mill products", "Manufacture of lumber and wood products, except furniture",
        "Manufacture of furniture and fixtures", "Manufacture of pulp, paper and paper products",
        "Printing and allied industries", "Manufacture of chemical and allied products",
        "Manufacture of petroleum and coal products", "Manufacture of plastic products, except otherwise classified",
        "Manufacture of rubber products", "Manufacture of leather tanning, leather products and fur skins",
        "Manufacture of ceramic, stone and clay products", "Manufacture of iron and steel",
        "Manufacture of non-ferrous metals and products", "Manufacture of fabricated metal products",
        "Manufacture of general-purpose machinery", "Manufacture of production machinery",
        "Manufacture of business oriented machinery", "Electronic parts, devices and electronic circuits",
        "Manufacture of electrical machinery, equipment and supplies", "Manufacture of information and communication electronics equipment",
        "Manufacture of transportation equipment", "Miscellaneous manufacturing industries"
    ],
    "Electricity, Gas, Heat supply and Water": ["Electricity, gas, heat supply and water"],
    "Information and communications": [
        "Information and communications", "Communications", "Broadcasting",
        "Information services", "Internet based services", 
        "Video picture, sound information, character information production and distribution"
    ],
    "Transport and postal activities": [
        "Transport and postal activities", "Railway transport", "Road passenger transport",
        "Road freight transport", "Water transport", "Air transport", "Warehousing",
        "Services incidental to transport", "Postal activities, including mail delivery"
    ],
    "Wholesale and retail trade": [
        "Wholesale and retail trade", "Wholesale trade", "Retail trade, general merchandise",
        "Retail trade (dry goods, apparel and apparel accessories)", "Retail trade (food and beverage)",
        "Machinery and equipment", "Miscellaneous retail trade"
    ],
    "Finance and Insurance": ["Finance and insurance"],
    "Real estate and goods rental and leasing": [
        "Real estate and goods rental and leasing", "Real estate", "Goods rental and leasing"
    ],
    "Scientific research, professional and technical services": [
        "Scientific research, professional and technical services", 
        "Scientific and development research institutes", "Professional services, N.E.C",
        "Advertising", "Technical services, N.E.C"
    ],
    "Accommodations, eating and drinking services": [
        "Accommodations, eating and drinking services", "Accommodations", 
        "Eating and drinking services", "Food take out and delivery services"
    ],
    "Living-related and personal services and amusement services": [
        "Living-related and personal services and amusement services", 
        "Laundry, beauty and bath services", "Miscellaneous living-related and personal services",
        "Services for amusement and hobbies"
    ],
    "Education, learning support": [
        "Education, learning support", "School education", "Miscellaneous education, learning support"
    ],
    "Medical, health care and welfare": [
        "Medical, health care and welfare", "Medical and other health services",
        "Public health and hygiene", "Social insurance, social welfare and care services"
    ],
    "Compound services": [
        "Compound services", "Postal services, except otherwise classified", "Cooperative associations, N.E.C"
    ],
    "Services": [
        "Services, N.E.C", "Waste disposal business", "Automobile maintenance services",
        "Machine, etc. repair services, except otherwise classified", "Employment and worker dispatching services",
        "Miscellaneous business services", "Political, business and cultural organizations",
        "Religion", "Miscellaneous services", "Foreign governments and international agencies in Japan",
        "Government, except elsewhere classified", "National Government services",
        "Local Government services", "Industries unable to classify","Agriculture and Forestry","Agriculture","Forestry","Non-agricultural industries","Fisheries","Fisheries, except aquaculture","Aquaculture"
    ]
}

# Transform the DataFrame
def regroup_industry(industry):
    for new_group, old_groups in industry_mapping.items():
        if industry in old_groups:
            return new_group
    return "Uncategorized"

df['Regrouped Industry'] = df['Industry_EN'].apply(regroup_industry)




In [14]:
df.head()
#value == 10,000

filtered_df = df[df['Organization_Type'] == '00']
employment_data = filtered_df[['Regrouped Industry', 'Value']]
employment_data.to_csv("japan_labor_2025_march.csv", index=False, encoding='utf-8')